#### Configuration

In [2]:
# ============================================================================
# CONFIGURATION - Modify these variables
# ============================================================================

# Lakehouse name (must be created in your workspace)
LAKEHOUSE_NAME = "Lakehouse_RawData"

# Excel file path in Lakehouse Files
# Upload your Excel file to: Lakehouse -> Files -> excel_files/
EXCEL_FILE_PATH = "/lakehouse/default/Files/Enterprise_Data_Model.xlsx"

# Schema name for tables
SCHEMA_NAME = "enterprise"

# Table prefix (optional, leave empty for no prefix)
TABLE_PREFIX = ""

# Overwrite existing tables
OVERWRITE_EXISTING = True

# Enable detailed logging
VERBOSE_MODE = True


StatementMeta(, dedf02ab-21d2-4384-ab96-67692606400a, 4, Finished, Available, Finished)

#### Setup and Imports

In [3]:
# Import required libraries
import pandas as pd
import openpyxl
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from datetime import datetime
import os

# Initialize Spark session
spark = SparkSession.builder.appName("ExcelToLakehouse").getOrCreate()

print("✓ Imports successful")
print(f"✓ Spark version: {spark.version}")
print(f"✓ Timestamp: {datetime.now()}")

StatementMeta(, dedf02ab-21d2-4384-ab96-67692606400a, 5, Finished, Available, Finished)

✓ Imports successful
✓ Spark version: 3.5.5.5.4.20251218.3
✓ Timestamp: 2026-02-11 12:23:38.425162


#### Helper Functions

In [4]:
def log(message, level="INFO"):
    """Custom logging function"""
    if VERBOSE_MODE or level == "ERROR":
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        print(f"[{timestamp}] {level}: {message}")

def clean_column_names(df):
    """Clean column names for Delta table compatibility"""
    for col in df.columns:
        # Replace spaces and special characters
        new_col = col.replace(" ", "_").replace("-", "_").replace("(", "").replace(")", "")
        new_col = new_col.replace("/", "_").replace(".", "_")
        if new_col != col:
            df = df.withColumnRenamed(col, new_col)
    return df

def get_table_name(sheet_name):
    """Generate table name from sheet name"""
    table_name = sheet_name.replace(" ", "_").replace("-", "_")
    if TABLE_PREFIX:
        return f"{TABLE_PREFIX}{table_name}"
    return table_name

def infer_spark_schema(df_pandas):
    """Convert pandas DataFrame to Spark DataFrame with proper schema"""
    return spark.createDataFrame(df_pandas)

print("✓ Helper functions loaded")

StatementMeta(, dedf02ab-21d2-4384-ab96-67692606400a, 6, Finished, Available, Finished)

✓ Helper functions loaded


### Step 1: Validate Excel File

In [5]:
log("Validating Excel file path...")

# Check if file exists
if not os.path.exists(EXCEL_FILE_PATH):
    log(f"ERROR: Excel file not found at {EXCEL_FILE_PATH}", "ERROR")
    log("Please upload your Excel file to Lakehouse -> Files -> excel_files/", "ERROR")
    raise FileNotFoundError(f"File not found: {EXCEL_FILE_PATH}")

log(f"✓ Excel file found: {EXCEL_FILE_PATH}")

# Get sheet names
wb = openpyxl.load_workbook(EXCEL_FILE_PATH, read_only=True)
sheet_names = wb.sheetnames
wb.close()

log(f"✓ Found {len(sheet_names)} sheets in Excel file")
for i, sheet_name in enumerate(sheet_names, 1):
    log(f"  {i}. {sheet_name}")

print(f"\n{'='*80}")
print(f"Total sheets to process: {len(sheet_names)}")
print(f"{'='*80}")

StatementMeta(, dedf02ab-21d2-4384-ab96-67692606400a, 7, Finished, Available, Finished)

[2026-02-11 12:23:39] INFO: Validating Excel file path...
[2026-02-11 12:23:39] INFO: ✓ Excel file found: /lakehouse/default/Files/Enterprise_Data_Model.xlsx
[2026-02-11 12:23:40] INFO: ✓ Found 13 sheets in Excel file
[2026-02-11 12:23:40] INFO:   1. DIM_Date
[2026-02-11 12:23:40] INFO:   2. DIM_Employees
[2026-02-11 12:23:40] INFO:   3. DIM_Products
[2026-02-11 12:23:40] INFO:   4. DIM_Customers
[2026-02-11 12:23:40] INFO:   5. DIM_Suppliers
[2026-02-11 12:23:40] INFO:   6. DIM_Documents
[2026-02-11 12:23:40] INFO:   7. DIM_Accounts
[2026-02-11 12:23:40] INFO:   8. DIM_Business_Lines
[2026-02-11 12:23:40] INFO:   9. FACT_Sales
[2026-02-11 12:23:40] INFO:   10. FACT_Purchases
[2026-02-11 12:23:40] INFO:   11. FACT_Budget
[2026-02-11 12:23:40] INFO:   12. FACT_Inventory
[2026-02-11 12:23:40] INFO:   13. FACT_HR

Total sheets to process: 13


### Step 2: Read and Analyze All Sheets

In [6]:
log("Reading all sheets from Excel file...")

# Dictionary to store all DataFrames
sheets_data = {}
sheets_metadata = {}

for sheet_name in sheet_names:
    try:
        log(f"Reading sheet: {sheet_name}")
        
        # Read Excel sheet into pandas DataFrame
        df_pandas = pd.read_excel(EXCEL_FILE_PATH, sheet_name=sheet_name)
        
        # Store metadata
        sheets_metadata[sheet_name] = {
            "row_count": len(df_pandas),
            "column_count": len(df_pandas.columns),
            "columns": list(df_pandas.columns)
        }
        
        # Convert to Spark DataFrame
        df_spark = infer_spark_schema(df_pandas)
        
        # Clean column names
        df_spark = clean_column_names(df_spark)
        
        # Store in dictionary
        sheets_data[sheet_name] = df_spark
        
        log(f"  ✓ Loaded {len(df_pandas)} rows, {len(df_pandas.columns)} columns")
        
    except Exception as e:
        log(f"ERROR reading sheet {sheet_name}: {str(e)}", "ERROR")
        continue

print(f"\n{'='*80}")
print(f"Successfully loaded {len(sheets_data)} sheets")

StatementMeta(, dedf02ab-21d2-4384-ab96-67692606400a, 8, Finished, Available, Finished)

[2026-02-11 12:23:40] INFO: Reading all sheets from Excel file...
[2026-02-11 12:23:40] INFO: Reading sheet: DIM_Date
[2026-02-11 12:23:42] INFO:   ✓ Loaded 1096 rows, 12 columns
[2026-02-11 12:23:42] INFO: Reading sheet: DIM_Employees
[2026-02-11 12:23:42] INFO:   ✓ Loaded 250 rows, 8 columns
[2026-02-11 12:23:42] INFO: Reading sheet: DIM_Products
[2026-02-11 12:23:42] INFO:   ✓ Loaded 500 rows, 10 columns
[2026-02-11 12:23:42] INFO: Reading sheet: DIM_Customers
[2026-02-11 12:23:43] INFO:   ✓ Loaded 1000 rows, 9 columns
[2026-02-11 12:23:43] INFO: Reading sheet: DIM_Suppliers
[2026-02-11 12:23:43] INFO:   ✓ Loaded 200 rows, 7 columns
[2026-02-11 12:23:43] INFO: Reading sheet: DIM_Documents
[2026-02-11 12:23:43] INFO:   ✓ Loaded 5000 rows, 5 columns
[2026-02-11 12:23:43] INFO: Reading sheet: DIM_Accounts
[2026-02-11 12:23:43] INFO:   ✓ Loaded 14 rows, 4 columns
[2026-02-11 12:23:43] INFO: Reading sheet: DIM_Business_Lines
[2026-02-11 12:23:43] INFO:   ✓ Loaded 6 rows, 4 columns
[2026-

### Step 3: Display Sheet Summaries

In [7]:
print("\n" + "="*80)
print("SHEET SUMMARIES")
print("="*80)

for sheet_name, metadata in sheets_metadata.items():
    print(f"\n📊 Sheet: {sheet_name}")
    print(f"   Rows: {metadata['row_count']:,}")
    print(f"   Columns: {metadata['column_count']}")
    print(f"   Column Names: {', '.join(metadata['columns'][:5])}")
    if metadata['column_count'] > 5:
        print(f"                 ... and {metadata['column_count'] - 5} more columns")
    
    # Show sample data
    if sheet_name in sheets_data:
        print(f"\n   Sample data:")
        sheets_data[sheet_name].show(3, truncate=False)

StatementMeta(, dedf02ab-21d2-4384-ab96-67692606400a, 9, Finished, Available, Finished)


SHEET SUMMARIES

📊 Sheet: DIM_Date
   Rows: 1,096
   Columns: 12
   Column Names: Date_ID, Date, Year, Quarter, Month
                 ... and 7 more columns

   Sample data:
+--------+-------------------+----+-------+-----+----------+----+---+-----------+--------+----------+----------+
|Date_ID |Date               |Year|Quarter|Month|Month_Name|Week|Day|Day_of_Week|Day_Name|Is_Weekend|Is_Holiday|
+--------+-------------------+----+-------+-----+----------+----+---+-----------+--------+----------+----------+
|20230101|2023-01-01 00:00:00|2023|1      |1    |January   |52  |1  |7          |Sunday  |1         |0         |
|20230102|2023-01-02 00:00:00|2023|1      |1    |January   |1   |2  |1          |Monday  |0         |0         |
|20230103|2023-01-03 00:00:00|2023|1      |1    |January   |1   |3  |2          |Tuesday |0         |0         |
+--------+-------------------+----+-------+-----+----------+----+---+-----------+--------+----------+----------+
only showing top 3 rows


📊 Sheet

### Step 4: Create Delta Tables in Lakehouse

In [8]:
log("Creating Delta tables in Lakehouse...")

# Track success/failure
tables_created = []
tables_failed = []

for sheet_name, df_spark in sheets_data.items():
    try:
        table_name = get_table_name(sheet_name)
        log(f"Creating table: {table_name}")
        
        # Write mode
        write_mode = "overwrite" if OVERWRITE_EXISTING else "error"
        
        # Write as Delta table
        df_spark.write \
            .format("delta") \
            .mode(write_mode) \
            .option("overwriteSchema", "true") \
            .saveAsTable(f"dbo.{table_name}")
        
        log(f"  ✓ Created table: {table_name} with {df_spark.count()} rows")
        tables_created.append(table_name)
        
    except Exception as e:
        log(f"ERROR creating table {table_name}: {str(e)}", "ERROR")
        tables_failed.append(table_name)
        continue

print(f"\n{'='*80}")
print(f"✓ Successfully created {len(tables_created)} tables")
if tables_failed:
    print(f"✗ Failed to create {len(tables_failed)} tables: {', '.join(tables_failed)}")
print(f"{'='*80}")

StatementMeta(, dedf02ab-21d2-4384-ab96-67692606400a, 10, Finished, Available, Finished)

[2026-02-11 12:24:06] INFO: Creating Delta tables in Lakehouse...
[2026-02-11 12:24:06] INFO: Creating table: DIM_Date
[2026-02-11 12:24:24] INFO:   ✓ Created table: DIM_Date with 1096 rows
[2026-02-11 12:24:24] INFO: Creating table: DIM_Employees
[2026-02-11 12:24:30] INFO:   ✓ Created table: DIM_Employees with 250 rows
[2026-02-11 12:24:30] INFO: Creating table: DIM_Products
[2026-02-11 12:24:36] INFO:   ✓ Created table: DIM_Products with 500 rows
[2026-02-11 12:24:36] INFO: Creating table: DIM_Customers
[2026-02-11 12:24:41] INFO:   ✓ Created table: DIM_Customers with 1000 rows
[2026-02-11 12:24:41] INFO: Creating table: DIM_Suppliers
[2026-02-11 12:24:46] INFO:   ✓ Created table: DIM_Suppliers with 200 rows
[2026-02-11 12:24:46] INFO: Creating table: DIM_Documents
[2026-02-11 12:24:53] INFO:   ✓ Created table: DIM_Documents with 5000 rows
[2026-02-11 12:24:53] INFO: Creating table: DIM_Accounts
[2026-02-11 12:24:58] INFO:   ✓ Created table: DIM_Accounts with 14 rows
[2026-02-11 12:

### Step 5: Verify Tables

In [9]:
log("Verifying created tables...")

print("\n" + "="*80)
print("TABLE VERIFICATION")
print("="*80)

for table_name in tables_created:
    try:
        # Read table
        df = spark.read.table(f"dbo.{table_name}")
        row_count = df.count()
        col_count = len(df.columns)
        
        print(f"\n✓ Table: {table_name}")
        print(f"  Rows: {row_count:,}")
        print(f"  Columns: {col_count}")
        print(f"  Schema: {', '.join([f'{field.name}' for field in df.schema.fields[:5]])}")
        if col_count > 5:
            print(f"          ... and {col_count - 5} more columns")
        
    except Exception as e:
        log(f"ERROR verifying table {table_name}: {str(e)}", "ERROR")

StatementMeta(, dedf02ab-21d2-4384-ab96-67692606400a, 11, Finished, Available, Finished)

[2026-02-11 12:25:32] INFO: Verifying created tables...

TABLE VERIFICATION

✓ Table: DIM_Date
  Rows: 1,096
  Columns: 12
  Schema: Date_ID, Date, Year, Quarter, Month
          ... and 7 more columns

✓ Table: DIM_Employees
  Rows: 250
  Columns: 8
  Schema: Employee_ID, Employee_Name, Department, Position, Location
          ... and 3 more columns

✓ Table: DIM_Products
  Rows: 500
  Columns: 10
  Schema: Product_ID, Product_Name, SKU, Category, Brand
          ... and 5 more columns

✓ Table: DIM_Customers
  Rows: 1,000
  Columns: 9
  Schema: Customer_ID, Customer_Name, Customer_Type, Customer_Segment, City
          ... and 4 more columns

✓ Table: DIM_Suppliers
  Rows: 200
  Columns: 7
  Schema: Supplier_ID, Supplier_Name, Supplier_Type, Country, Payment_Terms
          ... and 2 more columns

✓ Table: DIM_Documents
  Rows: 5,000
  Columns: 5
  Schema: Document_ID, Document_Number, Document_Type, Document_Date, Status

✓ Table: DIM_Accounts
  Rows: 14
  Columns: 4
  Schema: Accou

### Step 6: Query Sample Data from Each Table

In [10]:
# TEST Query: Dimension tables
print("\n" + "="*80)
print("SAMPLE QUERIES - DIMENSION TABLES")
print("="*80)

# DIM_Date
print("\n📅 DIM_Date - First 5 records:")
spark.sql(f"SELECT * FROM dbo.DIM_Date LIMIT 5").show(truncate=False)

# DIM_Employees
print("\n👥 DIM_Employees - Active employees:")
spark.sql(f"SELECT * FROM dbo.DIM_Employees WHERE Is_Active = 1 LIMIT 5").show(truncate=False)

# DIM_Products
print("\n📦 DIM_Products - Active products:")
spark.sql(f"SELECT Product_ID, Product_Name, Category, Brand, Unit_Price FROM dbo.DIM_Products WHERE Is_Active = 1 LIMIT 5").show(truncate=False)

StatementMeta(, dedf02ab-21d2-4384-ab96-67692606400a, 12, Finished, Available, Finished)


SAMPLE QUERIES - DIMENSION TABLES

📅 DIM_Date - First 5 records:
+--------+-------------------+----+-------+-----+----------+----+---+-----------+--------+----------+----------+
|Date_ID |Date               |Year|Quarter|Month|Month_Name|Week|Day|Day_of_Week|Day_Name|Is_Weekend|Is_Holiday|
+--------+-------------------+----+-------+-----+----------+----+---+-----------+--------+----------+----------+
|20230518|2023-05-18 00:00:00|2023|2      |5    |May       |20  |18 |4          |Thursday|0         |0         |
|20230519|2023-05-19 00:00:00|2023|2      |5    |May       |20  |19 |5          |Friday  |0         |0         |
|20230520|2023-05-20 00:00:00|2023|2      |5    |May       |20  |20 |6          |Saturday|1         |1         |
|20230521|2023-05-21 00:00:00|2023|2      |5    |May       |20  |21 |7          |Sunday  |1         |0         |
|20230522|2023-05-22 00:00:00|2023|2      |5    |May       |21  |22 |1          |Monday  |0         |0         |
+--------+-------------------+

In [11]:
# TEST Query: Query Fact tables
print("\n" + "="*80)
print("SAMPLE QUERIES - FACT TABLES")
print("="*80)

# FACT_Sales summary
print("\n💰 FACT_Sales - Summary by Date:")
spark.sql(f"""
SELECT 
    Date_ID,
    COUNT(*) as Transaction_Count,
    SUM(Total_Amount) as Total_Sales,
    SUM(Profit_Amount) as Total_Profit
FROM dbo.FACT_Sales
GROUP BY Date_ID
ORDER BY Date_ID DESC
LIMIT 10
""").show()

# FACT_Inventory current stock
print("\n📊 FACT_Inventory - Recent movements:")
spark.sql(f"""
SELECT 
    Product_ID,
    Movement_Type,
    SUM(Quantity) as Total_Quantity,
    COUNT(*) as Movement_Count
FROM dbo.FACT_Inventory
GROUP BY Product_ID, Movement_Type
LIMIT 10
""").show()

StatementMeta(, dedf02ab-21d2-4384-ab96-67692606400a, 13, Finished, Available, Finished)


SAMPLE QUERIES - FACT TABLES

💰 FACT_Sales - Summary by Date:
+--------+-----------------+------------------+------------------+
| Date_ID|Transaction_Count|       Total_Sales|      Total_Profit|
+--------+-----------------+------------------+------------------+
|20251231|               11|216353.77000000002|          120229.9|
|20251230|               10|         240776.36|          65668.31|
|20251229|               10|         244537.43|          85173.33|
|20251228|                7|         197632.99| 68406.20999999999|
|20251227|               15|         280735.59|          68479.15|
|20251226|                8|180345.36000000002|           34400.7|
|20251225|                8|         171460.88| 41234.78999999999|
|20251224|                9|         204925.49|          72603.61|
|20251223|               11|         290280.84| 90924.03999999998|
|20251222|               13|376684.01999999996|100193.61000000002|
+--------+-----------------+------------------+------------------+